### Chat with Data using Agents

In [9]:
import os
from dotenv import load_dotenv

# from langchain.agents import AgentType, initialize_agent, load_tools
# from langchain_community.llms import Ollama
from langchain_groq import ChatGroq

from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

from common_functions import generate_synthetic_data, \
	display_md, extract_quotes, datasets_dir  # ensure_ollama_running, 

load_dotenv()

# ensure_llama_running()
# llm_model = os.getenv('LLM_MODEL')
# llm = Ollama(model=llm_model)
llm = ChatGroq(model_name=os.getenv('GROQ_SMALL_MODEL'), temperature=1.0)
data_gen_llm = ChatGroq(model_name=os.getenv('GROQ_LARGE_MODEL'), temperature=1.0)

#### Generate synthetic data

In [10]:
topic = 'sales'
fields = ['product_id', 'sales', 'date']
examples = [{ 'example': 'product_id: 1, sales: 100, dates: any dates from 2020 to 2022' }]

data_save_path = os.path.join(datasets_dir, f'{topic}_synthetic_data.csv')
df = generate_synthetic_data(data_gen_llm, topic, rows=100, fields=fields,
								examples=examples, data_save_path=data_save_path)
df.head()

,product_id,sales,date
0,1,120,2020-01-05
1,2,80,2020-01-10
2,3,150,2020-01-15
3,1,90,2020-02-12
4,2,70,2020-02-18


#### Define a chain for asking queries

In [11]:
prompt = f'''
Act as a Data Analyst and Data Scientist for a company that uses {topic} data.
Answer the user's question using the {topic} data below. Make useful text bold or italic according to the context.

```{df.to_csv(index=False)}```
'''

chatPromptTemplate = ChatPromptTemplate(
	messages = [
		SystemMessage(prompt),
		HumanMessagePromptTemplate.from_template('{question}'),
	],
	input_variables=['query_context', 'question'],
)

chain = LLMChain(
	llm=llm,
	prompt=chatPromptTemplate,
	verbose=False,
	output_key='query_answer',
)

def ask_query(query, display=False, return_quoted=False):
	response = chain.invoke(
		{ "question": query },
		output_key="query_answer",
	)
	answer = response["query_answer"]
	if return_quoted:
		return extract_quotes(answer)
	if display:
		display_md(answer)
	else:
		return answer

In [12]:
# ask_query("What is the total sales for product 1?", display=True)

In [13]:
# ask_query("On which months are overal sales higher?", display=True)

In [14]:
# ask_query("What are the possible reasons for the drop in sales sometimes?", display=True)

#### Chatting with CSV (Similar to RAG over SQL)

In [ ]:
# create a code interpreter agent that can execute code over the data including plotting graphs

from codeinterpreterapi import CodeInterpreterSession, File

with CodeInterpreterSession(llm) as session:
	response = session.generate_response(
		user_msg = 'Plot a graph to represent sales over the years for each product.' \
					'Use columns: ' + ', '.join(df.columns),
		files = [File.from_path(data_save_path)],
	)
	response.show()